In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
import pickle

# Load the dataset
dataset = pd.read_csv("C:/Users/bhatk/OneDrive/Documents/jupyter/diabetes_data_prediction/diabetes_prediction_dataset.csv")

# Check for missing values
if dataset.isnull().sum().sum() > 0:
    dataset = dataset.dropna()

# Encode categorical features if any
categorical_columns = dataset.select_dtypes(include=['object']).columns
for col in categorical_columns:
    dataset[col] = dataset[col].astype('category').cat.codes

# Split features and target
X = dataset.drop("diabetes", axis=1)
y = dataset["diabetes"]

# Handle imbalanced data using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Scale the data
scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Save the model and scaler using pickle
with open('diabetes_model.pkl', 'wb') as model_file:
    pickle.dump(clf, model_file)

with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

# Predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nROC-AUC Score:", roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

# Test with custom input values (ensure the same scaling is applied)
custom_input = np.array([[1,	66.0,	0,	0	,3,	27.83,	5.7,	155	]])  # Replace with actual input values

# Load the scaler and model for prediction
with open('scaler.pkl', 'rb') as scaler_file:
    loaded_scaler = pickle.load(scaler_file)

with open('diabetes_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

custom_input_scaled = loaded_scaler.transform(custom_input)
custom_prediction = loaded_model.predict(custom_input_scaled)
print("Custom Input Prediction:", "Diabetic" if custom_prediction[0] == 1 else "Non-Diabetic")


Confusion Matrix:
[[17853   440]
 [  481 17826]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97     18293
           1       0.98      0.97      0.97     18307

    accuracy                           0.97     36600
   macro avg       0.97      0.97      0.97     36600
weighted avg       0.97      0.97      0.97     36600


ROC-AUC Score: 0.9971009819879605
Custom Input Prediction: Non-Diabetic


C:\Users\bhatk\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'diabetes_model.pkl'